In [ ]:
%load_ext autoreload
%autoreload 2

Procedure

- Create mesh
    - Generates object with nodes and elements as properties
    - Depends on coordinate system, mesh size, and symmetry conditions
- Add structures 
    - 

In [ ]:
import numpy as np

- Check get_plane_node_ids to see if 0s are necessary
- Fix formatting of numbers in material cards
- Fix pml/struct interface with 4 node overlap 
    - Check len(pml_and_struct_nodes) condition is valid
- Add validation functions
    - Nodes exist on center axis regardless of mesh symmetry so peak arf load is modeled
    - Mesh extent minima are less than maxima
- Add card dictionary/dataclass to contain all dyna cards and defaults
    - Materials
    - Load curves
    - Loads (nodal forces list)
    - Database
    - Master keyword file
- Add nested and unnested folder structure write out options
    - Unnested: all files in same folder for single simulation
    - Nested: folder structure for batch simulations:
        project/mesh.dyn
               /index file 
               /load_folders/arf_load.dyn
                            /material_folders/materials.dyn
                                             /master.dyn
    - Create reader/writer for loading the index file and extracting simulation data. Use generic folder names (arf_load_x, material_x) and keep variable parameters associated with folder x in index file

- Create mesh 
    - Depends on element size and extents
    - Returns coordinate axes (size of each is nx, ny, nz)
- Add structures to mesh
    - Get nodes in struct based on coordinates
    - Change part id of elements in struct
        - All nodes in element should be within struct? 
    -- Generate list of node ids in struct
- Add boundary constraints 
    - Depends on mesh symmetry
    - Change tc and rc of nodes on faces and edges
- Add pml
    - Depends on mesh symmetry and pml thickness
    - Change part id of non-symmetry plane elements
    - Part id + 1 if also in struct
- Finish dyna deck
    - Add material
    - Add load curves
    - Add arf load based on field params (pre-generated)
    - Add database/control parameters
        - Node sets?
- Validation functions
    - Make sure nodes exist on center axis regardless of mesh symmetry so peak arf load is modeled



In [ ]:
# @dataclass
# class Extent:
#     x: Tuple[float, float]
#     y: Tuple[float, float]
#     z: Tuple[float, float]

# @dataclass
# class Coordinates:
#     nx: int
#     ny: int
#     nz: int
#     extent: Extent
#     x: np.ndarray = field(init=False, repr=False)
#     y: np.ndarray = field(init=False, repr=False)
#     z: np.ndarray = field(init=False, repr=False)

#     def __post_init__(self):
#         self.x = np.linspace(self.extent.x[0], self.extent.x[1], self.nx)
#         self.y = np.linspace(self.extent.y[0], self.extent.y[1], self.ny)
#         self.z = np.linspace(self.extent.z[0], self.extent.z[1], self.nz)


In [ ]:
from fem.dyna.mesh import Coordinates, DynaMesh
from fem.dyna._structure import Structure
from fem.dyna.material import KelvinMaxwellViscoelastic

mat = KelvinMaxwellViscoelastic(density=1, E=26.11, nu=0.499, eta=2.34)
mat2 = KelvinMaxwellViscoelastic(density=1, E=2*26.11, nu=0.499, eta=2.34)

struct_list = [
    Structure(
        shape = 'rectangle', 
        args = [-1, -0.5, -0.5, -0, -4, -2],
        material = mat2,  
    ), 
    Structure(
        shape = 'sphere', 
        args = [0, 0, -2, 1],
        material = mat2,  
    ), 
]

coords = Coordinates(
    nx=11, ny=11, nz=11,
    xmin=-1.0, xmax=0.0,
    ymin=-1.0, ymax=0.0,
    zmin=-4.0, zmax=0.0
)
coords.x
symmetry = 'q'
mesh = DynaMesh(coords, symmetry, mat)

mesh.constrain_boundary_nodes()
mesh.add_pml(pml_thickness=2)
mesh.add_struct_list(struct_list)

mesh.set_control(end_time=4.5e-3)
mesh.set_database(dt=2e-5)
mesh.set_master(title='testing')

project_path = r"C:\Users\joeyr\repos\fem\sims"
load_folder_name = 'load_0'
material_folder_name = 'material_0'
mesh.write_all_dyna_cards(project_path, load_folder_name, material_folder_name)


In [ ]:
from fem.dyna.mesh import Coordinates, DynaMesh
from fem.dyna._structure import Structure
from fem.dyna.material import KelvinMaxwellViscoelastic

mat = KelvinMaxwellViscoelastic(density=1, E=26.11, nu=0.499, eta=2.34)
mat2 = KelvinMaxwellViscoelastic(density=1, E=2*26.11, nu=0.499, eta=2.34)

struct_list = [
    Structure(
        shape = 'rectangle', 
        args = [-1, -0.5, -0.5, -0, -4, -2],
        material = mat2,  
    ), 
    Structure(
        shape = 'sphere', 
        args = [0, 0, -2, 1],
        material = mat2,  
    ), 
]

coords = Coordinates(
    nx=11, ny=11, nz=11,
    xmin=-1.0, xmax=0.0,
    ymin=-1.0, ymax=0.0,
    zmin=-4.0, zmax=0.0
)
coords.x
symmetry = 'q'
mesh = DynaMesh(coords, symmetry, mat)

mesh.constrain_boundary_nodes()
mesh.add_pml(pml_thickness=2)
mesh.add_struct_list(struct_list)

mesh.set_control(end_time=4.5e-3)
mesh.set_database(dt=2e-5)
mesh.set_master(title='testing')

project_path = r"C:\Users\joeyr\repos\fem\sims"
load_folder_name = 'load_0'
material_folder_name = 'material_0'
mesh.write_all_dyna_cards(project_path, load_folder_name, material_folder_name)


In [ ]:
from scipy.io import loadmat
import pathlib



In [ ]:
print(pathlib.__file__)

In [38]:
import fem

print(fem.__file__)
p = pathlib.Path(fem.__file__).parents[1].joinpath('loads')
print(p)
print(list(p.glob('*')))

C:\Users\joeyr\repos\fem\fem\__init__.py
C:\Users\joeyr\repos\fem\loads
[WindowsPath('C:/Users/joeyr/repos/fem/loads/ArfLoad.dyn'), WindowsPath('C:/Users/joeyr/repos/fem/loads/f2dout.mat')]


In [ ]:
load_path = pathlib.Path(fem.__file__).parents[1].joinpath('loads')
mat_file = load_path / 'f2dout.mat'
mat = loadmat(mat_file)

mat

In [41]:
load_path = pathlib.Path(fem.__file__).parents[1].joinpath('loads')
mat_file = load_path / 'f2dout.mat'
mat = loadmat(mat_file)

mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sat Mar 18 18:10:26 2023',
 '__version__': '1.0',
 '__globals__': [],
 'FIELD_PARAMS': array([[(array([[ 1.00000e+00, -1.00000e+00,  0.00000e+00, -4.35000e+00,
                  7.00000e+00],
                [ 2.00000e+00, -9.75000e-01,  0.00000e+00, -4.35000e+00,
                  7.00000e+00],
                [ 3.00000e+00, -9.50000e-01,  0.00000e+00, -4.35000e+00,
                  7.00000e+00],
                ...,
                [ 2.80438e+05, -5.00000e-02,  9.75000e-01, -1.00000e-01,
                  7.00000e+00],
                [ 2.80439e+05, -2.50000e-02,  9.75000e-01, -1.00000e-01,
                  7.00000e+00],
                [ 2.80440e+05,  0.00000e+00,  9.75000e-01, -1.00000e-01,
                  7.00000e+00]]), array([[ 0.     , -0.01   ,  0.0435 ],
                [ 0.     , -0.00975,  0.0435 ],
                [ 0.     , -0.0095 ,  0.0435 ],
                ...,
                [ 0.00975, -0.0005 

In [ ]:
intensity = mat['intensity']
field_params = mat['FIELD_PARAMS']
mpn = field_params['measurementPointsandNodes'][0,0]

print(mpn)

# print(field_params.dtype)
# print(field_params['measurementPointsandNodes'])
# print(field_params['measurementPointsandNodes'].shape)
# print(field_params['measurementPointsandNodes'][0,0].shape)
# print(field_params['measurementPointsandNodes'][0,0])

# print(intensity.shape)
# print(field_params.dtype) 
# print(field_params['measurementPointsandNodes'][0,0].shape)

In [ ]:
# annas cell :)


### Load curves

In [60]:
def format_dyna_scientific(s):
    s = f"{s:>20.4E}"
    if 'E+' in s:
        s = f" {s.replace('E+0', 'E+')}" 
    elif 'E-' in s:
        s = f" {s.replace('E-0', 'E-')}"
    return s

t_arf = 70e-6
dt = 5e-6
tracks_between = 3
prf = 10e3
t_delay = (tracks_between + 1)/prf
n_arf = 1

t = []
for iarf in range(n_arf):
    narf_delay = iarf*t_delay
    t.extend([
        [narf_delay, 0],
        [narf_delay+dt, 1],
        [narf_delay+dt+t_arf, 1],
        [narf_delay+2*dt+t_arf, 0],
    ])

for a, o in t:
    s = format_dyna_scientific(a) + format_dyna_scientific(o)
    print(s)


           0.0000E+0           0.0000E+0
           5.0000E-6           1.0000E+0
           7.5000E-5           1.0000E+0
           8.0000E-5           0.0000E+0
